In [1]:
import methylprep
import methylcheck
from pathlib import Path
import logging
import requests 
import shutil
from tqdm.auto import tqdm
import gzip
import shutil

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel( logging.DEBUG )

In [2]:
def test_it(geo_id, working):
    # second, see if the series_matrix file exists.
    series_server = "https://ftp.ncbi.nlm.nih.gov"
    series_path = f"geo/series/{geo_id[:-3]}nnn/{geo_id}/matrix"
    file_name = f"{geo_id}_series_matrix.txt.gz"
    series_matrix_path = f"{series_server}/{series_path}/{file_name}"    
    series_matrix_path2 = f"{series_server}/geo/series/{geo_id[:-3]}nnn/matrix/{geo_id}/{file_name}"    
    # some series paths include the GPL platform code too, if multiple platforms are present
    series_matrix_path3 = f"{series_server}/{series_path}/{geo_id}-GPL21145_series_matrix.txt.gz"
    series_matrix_path4 = f"{series_server}/{series_path}/{geo_id}-GPL13534_series_matrix.txt.gz"
    try:
        saved_file = file_name.replace(' ','_')
        response = requests.head(series_matrix_path)
        response2 = requests.head(series_matrix_path2)
        response3 = requests.head(series_matrix_path3)
        response4 = requests.head(series_matrix_path4)
        if response.status_code == 200:
            url = series_matrix_path
        elif response2.status_code == 200:
            url = series_matrix_path2
        elif response3.status_code == 200:
            url = series_matrix_path3
        elif response4.status_code4 == 200:
            url = series_matrix_path4            
        else:
            print(response.status_code)
            url = None
        if url != None:
            print(f"Downloading {url}")
            # Streaming, so we can iterate over the response.
            with requests.get(url, stream=True) as r:
                total_size_in_bytes= int(r.headers.get('content-length', 0))
                block_size = 1024 #1 Kibibyte
                progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
                with open(Path(working.name, saved_file), 'wb') as f:
                    #shutil.copyfileobj(r.raw, f) --- needed when saving to EFS
                    for data in r.iter_content(block_size):
                        progress_bar.update(len(data))
                        f.write(data)
                progress_bar.close()                    
            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("ERROR, something went wrong")                    
            else:
                print(f"Download {saved_file} complete")               
                downloaded_files = True
    except Exception as e:
        import traceback
        print(traceback.format_exc())
        print(f"Download failed: {e}, trying other saved files")
        pass 
    print('testing series matrix')
    #return Path(geo_id,saved_file)
    saved_file_path = Path(geo_id,saved_file)
    if url == None or saved_file_path.exists() == False:
        print("no series_matrix file downloaded")
    elif '.gz' in Path(saved_file_path).suffixes:
        unzipped_file = str(saved_file_path)[:-3]
        with gzip.open(saved_file_path, 'rb') as f_in:
            with open(unzipped_file, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)    
        data = methylcheck.read_geo_processed.read_series_matrix(unzipped_file, include_headers_df=True)
        print(geo_id, data['df'].shape, data['headers_df'].head())
        print('OK')

In [3]:
#geo_ids = ['GSE128401'] # ['GSE103498','GSE75073']
# GSE128864 has both arrays, and failed in beta bake
# GSE132203 has no idats but has meta data to parse -- but betas are missing from series_matrix
# has sentrix_ids in 'title' column. super fast to run because no betas.
#for geo_id in geo_ids:
#    working = Path(geo_id)
#    test_it(geo_id, working)

extracted a bunch of IDS this way, from cut-n-pasted text

    geo_ids = [re.match('GSE\d+',i).group(0) for i in yy if re.match('GSE\d+',i)]

In [4]:
# DONE 'GSE75073', 'GSE81939', 'GSE86831',
geo_ids =  ['GSE86833', 'GSE89895', 'GSE90867', 'GSE90933', 'GSE90934', 'GSE92462', 'GSE92469', 'GSE92579', 'GSE92580', 'GSE92608', 'GSE93356', 'GSE93373', 'GSE94677', 'GSE94987', 'GSE95232', 'GSE95460', 'GSE95462', 'GSE95505', 'GSE95531', 'GSE98990', 'GSE99505', 'GSE99788', 'GSE100184', 'GSE100208', 'GSE100209', 'GSE100779', 'GSE100780', 'GSE100781', 'GSE100782', 'GSE100783', 'GSE100784', 'GSE100825', 'GSE100850', 'GSE101908', 'GSE102031', 'GSE103189', 'GSE103271', 'GSE103279', 'GSE103280', 'GSE103286', 'GSE103287', 'GSE103328', 'GSE103332', 'GSE103498', 'GSE103505', 'GSE103541', 'GSE103556', 'GSE103558', 'GSE103738', 'GSE104210', 'GSE106727', 'GSE106728', 'GSE107080', 'GSE107082', 'GSE108202', 'GSE108982', 'GSE108985', 'GSE109330', 'GSE109364', 'GSE109399', 'GSE109402', 'GSE109403', 'GSE109541', 'GSE110184', 'GSE110185', 'GSE110274', 'GSE110454', 'GSE110530', 'GSE110544', 'GSE110554', 'GSE110555', 'GSE110600', 'GSE111165', 'GSE111183', 'GSE111336', 'GSE111541', 'GSE111631', 'GSE111807', 'GSE112067', 'GSE112179', 'GSE112525', 'GSE112596', 'GSE112611', 'GSE112618', 'GSE112905', 'GSE112924', 'GSE113372', 'GSE113545', 'GSE113687', 'GSE114163', 'GSE114210', 'GSE114721', 'GSE114763', 'GSE114989', 'GSE115246', 'GSE115278', 'GSE115382', 'GSE115413', 'GSE115508', 'GSE115777', 'GSE116298', 'GSE116339', 'GSE116402', 'GSE116699', 'GSE116992', 'GSE117130', 'GSE117528', 'GSE117860', 'GSE117861', 'GSE117903', 'GSE117904', 'GSE117926', 'GSE118003', 'GSE118144', 'GSE118694', 'GSE118970', 'GSE118972', 'GSE119079', 'GSE119100', 'GSE119260', 'GSE119548', 'GSE119652', 'GSE120122', 'GSE120428', 'GSE120558', 'GSE121056', 'GSE121377', 'GSE121433', 'GSE121434', 'GSE122038', 'GSE122078', 'GSE122086', 'GSE122126', 'GSE122148', 'GSE122288', 'GSE122408', 'GSE122469', 'GSE122546', 'GSE122808', 'GSE122809', 'GSE122920', 'GSE122994', 'GSE123140', 'GSE123185', 'GSE123211', 'GSE123367', 'GSE123609', 'GSE123610', 'GSE123830', 'GSE123885', 'GSE123914', 'GSE124568', 'GSE124569', 'GSE124617', 'GSE124632', 'GSE124633', 'GSE125367', 'GSE125386', 'GSE125450', 'GSE126043', 'GSE126045', 'GSE126528', 'GSE126545', 'GSE126646', 'GSE126672', 'GSE128032', 'GSE128125', 'GSE128126', 'GSE128127', 'GSE128128', 'GSE128130', 'GSE128218', 'GSE128401', 'GSE128684', 'GSE128685', 'GSE128686', 'GSE128784', 'GSE128795', 'GSE128821', 'GSE129376', 'GSE129428', 'GSE129789', 'GSE130051', 'GSE130093', 'GSE130295', 'GSE130357', 'GSE130641', 'GSE130748', 'GSE130844', 'GSE130849', 'GSE131177', 'GSE131280', 'GSE131295', 'GSE131433', 'GSE131482', 'GSE131945', 'GSE131973', 'GSE132181', 'GSE132203', 'GSE132219', 'GSE132399', 'GSE132406', 'GSE132513', 'GSE132514', 'GSE132547', 'GSE132616', 'GSE132617', 'GSE132650', 'GSE132690', 'GSE132804', 'GSE132845', 'GSE132866']

In [ ]:
def test_pipeline_find_betas_any_source():
    import os;print(os.getcwd())
    for geo_id in geo_ids:
        kwargs = {'project_name': geo_id, 'data_dir': f'example_data/{geo_id}', 'clean':False, 'compress':False, 'verbose':True}
        result = methylprep.download.pipeline_find_betas_any_source(**kwargs)
        print(result)
test_pipeline_find_betas_any_source()

INFO:methylprep.download.geo:Your command line inputs: {'project_name': 'GSE86833', 'data_dir': 'example_data/GSE86833', 'clean': False, 'compress': False, 'verbose': True}
INFO:methylprep.download.geo:Starting batch GEO pipeline processor for ['GSE86833'].
INFO:methylprep.download.geo:Searching GEO NIH for GSE86833
INFO:methylprep.download.geo:http://www.ncbi.nlm.nih.gov/geo/browse/?view=series&display=500&zsort=date&search=+GSE86833


/Volumes/LEGX/GEO


INFO:methylprep.download.geo:1 series
Checking for idats: 100%|██████████| 2/2 [00:02<00:00,  1.16s/it]
INFO:methylprep.download.geo:example_data/GSE86833/tmpbt0sdtst/geo_alert GSE86833.csv written
INFO:methylprep.download.geo:0: {'title': 'Critical evaluation of the Illumina MethylationEPIC BeadChip microarray for whole-genome DNA methylation profiling', 'GSE': 'GSE86833', 'url': 'http://www.ncbi.nlm.nih.gov//geo/query/acc.cgi?acc=GSE86833', 'samples': 39, 'date': '2016-09-12', 'platform': 'GPL21145', 'idats': '1', 'file_name_0': 'GSE86833_RAW.tar', 'file_size_0': '677.4 Mb', 'file_link_0': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE86833&format=file', 'file_name_1': '', 'file_size_1': '', 'file_link_1': '', 'file_name_2': '', 'file_size_2': '', 'file_link_2': ''}
INFO:methylprep.download.geo:IDATs exist for GSE86833
INFO:methylprep.download.geo:Downloading GSE86833_family.xml
INFO:methylprep.download.geo:Unpacking GSE86833_RAW.tar                                              

example_data/GSE86833 example_data/GSE86833/tmpbt0sdtst
{'error': None, 'filenames': [], 'tempdir': <TemporaryDirectory 'example_data/GSE86833/tmpbt0sdtst'>}


INFO:methylprep.download.geo:1 series
Checking for idats: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it]
INFO:methylprep.download.geo:example_data/GSE89895/tmptc3q7l8l/geo_alert GSE89895.csv written
INFO:methylprep.download.geo:0: {'title': 'Impact of regulatory variation across human iPSCs and differentiated cells', 'GSE': 'GSE89895', 'url': 'http://www.ncbi.nlm.nih.gov//geo/query/acc.cgi?acc=GSE89895', 'samples': 220, 'date': '2016-11-17', 'platform': 'GPL21145', 'idats': '1', 'file_name_0': 'GSE89895_RAW.tar', 'file_size_0': '974.1 Mb', 'file_link_0': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE89895&format=file', 'file_name_1': '', 'file_size_1': '', 'file_link_1': '', 'file_name_2': '', 'file_size_2': '', 'file_link_2': ''}
INFO:methylprep.download.geo:IDATs exist for GSE89895
INFO:methylprep.download.geo:Downloading GSE89895_family.xml
INFO:methylprep.download.geo:Unpacking GSE89895_RAW.tar                                                 
INFO:methylprep.download.geo:Downlo

...

example_data/GSE115508 example_data/GSE115508/tmp1v7q4t7i
{'error': None, 'filenames': [], 'tempdir': <TemporaryDirectory 'example_data/GSE115508/tmp1v7q4t7i'>}


INFO:methylprep.download.geo:1 series
Checking for idats: 100%|██████████| 2/2 [00:01<00:00,  1.13it/s]
INFO:methylprep.download.geo:example_data/GSE115777/tmp78ahu8y6/geo_alert GSE115777.csv written
INFO:methylprep.download.geo:0: {'title': 'Comparative global methylation profile in DMS114 cells before and after acquired resistance to PD173074', 'GSE': 'GSE115777', 'url': 'http://www.ncbi.nlm.nih.gov//geo/query/acc.cgi?acc=GSE115777', 'samples': 4, 'date': '2018-11-01', 'platform': 'GPL21145', 'idats': '0', 'file_name_0': 'GSE115777_RAW.tar', 'file_size_0': '161.2 Mb', 'file_link_0': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE115777&format=file', 'file_name_1': 'Raw data included within Sample table', 'file_size_1': '', 'file_link_1': '', 'file_name_2': 'Processed data included within Sample table', 'file_size_2': '', 'file_link_2': ''}
INFO:methylprep.download.geo:Downloading https://ftp.ncbi.nlm.nih.gov/geo/series/GSE115nnn/GSE115777/matrix/GSE115777_series_matrix.txt.gz
GSE

{'error': None, 'filenames': [], 'tempdir': <TemporaryDirectory 'example_data/GSE115777/tmp78ahu8y6'>}


INFO:methylprep.download.geo:1 series
Checking for idats: 100%|██████████| 2/2 [00:01<00:00,  1.12it/s]
INFO:methylprep.download.geo:example_data/GSE116298/tmpaqh4fwx3/geo_alert GSE116298.csv written
INFO:methylprep.download.geo:0: {'title': 'Intra-Tumor DNA Methylation Heterogeneity in Glioblastoma; Implications for DNA Methylation-based Classification', 'GSE': 'GSE116298', 'url': 'http://www.ncbi.nlm.nih.gov//geo/query/acc.cgi?acc=GSE116298', 'samples': 47, 'date': '2019-02-27', 'platform': 'GPL21145', 'idats': '1', 'file_name_0': 'GSE116298_RAW.tar', 'file_size_0': '822.0 Mb', 'file_link_0': 'https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE116298&format=file', 'file_name_1': 'Raw data provided as supplementary file', 'file_size_1': '', 'file_link_1': '', 'file_name_2': 'Processed data included within Sample table', 'file_size_2': '', 'file_link_2': ''}
INFO:methylprep.download.geo:IDATs exist for GSE116298
INFO:methylprep.download.geo:Downloading GSE116298_family.xml
INFO:methylpr

> /Users/mmaxmeister/methylprep/methylprep/download/geo.py(150)geo_download()
    148                 LOGGER.info(f"[Attempt #{attempt}]")
    149             if attempt == 5:
--> 150                 LOGGER.error(f"Could not download after {attempt} attempts. Aborting.")
    151                 break
    152 

